In [11]:
import os
import sys
import pydicom
import numpy as np
import pandas as pd
import shutil
import glob
import cv2
import nibabel as nib

from PIL import Image 

In [12]:
"""
Routines for scaling biomedical image data by window level and width.
"""

win_dict = {'abdomen':
            {'wl': 60, 'ww': 400},
            'angio':
            {'wl': 300, 'ww': 600},
            'bone':
            {'wl': 300, 'ww': 1500},
            'brain':
            {'wl': 40, 'ww': 80},
            'chest':
            {'wl': 40, 'ww': 400},
            'lungs':
            {'wl': -400, 'ww': 1500}}

def win_scale(data, wl, ww, dtype, out_range):
    """
    Scale pixel intensity data using specified window level, width, and intensity range.
    """
    
    data_new = np.empty(data.shape, dtype=np.double)
    data_new.fill(out_range[1]-1)
    
    data_new[data <= (wl-ww/2.0)] = out_range[0]
    data_new[(data>(wl-ww/2.0))&(data<=(wl+ww/2.0))] = \
         ((data[(data>(wl-ww/2.0))&(data<=(wl+ww/2.0))]-(wl-0.5))/(ww-1.0)+0.5)*(out_range[1]-out_range[0])+out_range[0]
    data_new[data > (wl+ww/2.0)] = out_range[1]-1
    
    return data_new.astype(dtype)
        
def ct_win(im, wl, ww, dtype, out_range):    
    """
    Scale CT image represented as a `pydicom.dataset.FileDataset` instance.
    """

    # Convert pixel data from Houndsfield units to intensity:
    intercept = int(im[(0x0028, 0x1052)].value)
    slope = int(im[(0x0028, 0x1053)].value)
    data = (slope*im.pixel_array+intercept)

    # Scale intensity:
    return win_scale(data, wl, ww, dtype, out_range)

def dcm_2_jpg(dcm_path):
    # Extracting The Pixel Array

    ds = pydicom.dcmread(dcm_path)
    new_image = ds.pixel_array.astype(float)

    # Rescaling the image

    scaled_image = (np.maximum(new_image, 0) / new_image.max()) * 255.0
    # scaled_image = win_scale(scaled_image, 0, 255, float, [0, 600])

    # Displaying And Saving The Image

    scaled_image = np.uint8(scaled_image)
    final_image = np.array(scaled_image)
    # final_image = Image.fromarray(scaled_image)
    
    if ds.PhotometricInterpretation == 'MONOCHROME1':
        final_image = 255-final_image
    
    return final_image

def bit_operation_s0(board: np.array, logo: np.array):
        '''현재 앵글에 대한 위 표본 시각화 단계 1
        '''
        rows, cols, _ = logo.shape
        roi = board[0:rows, 0:cols]

        sample_logo_gray = cv2.cvtColor(logo, cv2.COLOR_BGR2GRAY)
        ret, mask = cv2.threshold(sample_logo_gray, 10, 255, cv2.THRESH_BINARY)
        mask_inv = cv2.bitwise_not(mask)

        board_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)

        sample_logo_fg = cv2.bitwise_and(logo, logo, mask=mask)

        dst = cv2.add(board_bg, sample_logo_fg)
        board[0:rows, 0:cols] = dst
    
        return board
    
def convert_gray2rgb(arr):
    width, height = arr.shape

    out = np.empty((width, height, 3), dtype=np.uint8)
    out[:,:,0] = arr
    out[:,:,1] = arr
    out[:,:,2] = arr

    return out

In [13]:
# file path glob
# covid_path = glob.glob(os.path.join('..', 'COVID-19', 'DCM', 'abnormal', '*'))

lung_abn_dcm_path = glob.glob(os.path.join('..', 'DCM', 'abnormal', '*.dcm'))
lung_abn_nii_path = glob.glob(os.path.join('..', 'nifti', '*.nii'))

lung_n_dcm_path = glob.glob(os.path.join('..', 'DCM', 'normal', '*.dcm'))


In [14]:
file_path = os.path.join('..', 'JPG', 'abnormal')

if not os.path.exists(file_path):
    os.mkdir(file_path)
    
if not os.path.exists(os.path.join(file_path, 'img_roi')):
    os.mkdir(os.path.join(file_path, 'img_roi'))
    
if not os.path.exists(os.path.join(file_path, 'img')):
    os.mkdir(os.path.join(file_path, 'img'))

if not os.path.exists(os.path.join(file_path, 'roi')):
    os.mkdir(os.path.join(file_path, 'roi'))

for dcm_path in lung_abn_dcm_path:
    f_name = dcm_path.split('\\')[-1]
    name, ext = os.path.splitext(f_name)
    
    img = dcm_2_jpg(dcm_path)
    
    new_img = convert_gray2rgb(img)
    
    for nii_path in lung_abn_nii_path:
        if name in nii_path:
            proxy = nib.load(nii_path)
            roi = np.array(proxy.get_fdata()).T[0,:,:]
            
            new_roi = convert_gray2rgb(roi)
            
            new_roi = cv2.cvtColor(new_roi, cv2.COLOR_BGR2RGB)

            bgr = new_roi[:,:,0:3]
            mask = cv2.inRange(bgr, (0,0,0), (150,150,150))
            bgr_new = bgr.copy()
            bgr_new[mask!=255] = (50,205,50)
            
            new_img = bit_operation_s0(board=new_img, logo=bgr_new)
    
    # img+roi
    cv2.imwrite(os.path.join(file_path, 'img_roi', f'{name}.jpg'), new_img)
    # img
    cv2.imwrite(os.path.join(file_path, 'img', f'{name}_src.jpg'), img)
    # roi
    cv2.imwrite(os.path.join(file_path, 'roi', f'{name}_roi.jpg'), new_roi)

In [34]:

# proxy = nib.load(lung_abn_nii_path[0])
# roi = np.array(proxy.get_fdata()).T[0,:,:]

# new_roi = convert_gray2rgb(roi)

# new_roi = cv2.cvtColor(new_roi, cv2.COLOR_BGR2RGB)

# bgr = new_roi[:,:,0:3]

# mask = cv2.inRange(bgr, (0,0,0), (150,150,150))

# bgr_new = bgr.copy()

# bgr_new[mask!=255] = (50,205,50)

# cv2.imwrite(os.path.join('..', 'sample111.jpg'), bgr_new)


# # print(np.unique(new_roi))

# # cv2.imshow('aa', new_roi)
# # cv2.waitKey()
# # cv2.destroyAllWindows()

True

In [ ]:
# file_path = os.path.join('..', 'JPG', 'normal')

# if not os.path.exists(file_path):
#     os.mkdir(file_path)

# for dcm_path in lung_n_dcm_path:
#     f_name = dcm_path.split('\\')[-1]
#     name, ext = os.path.splitext(f_name)
    
#     img = dcm_2_jpg(dcm_path)
    
#     cv2.imwrite(os.path.join(file_path, f'{name}.jpg'), img)

In [ ]:
# # sample_dcm_path = lung_n_dcm_path[-1]

# # f_name = sample_dcm_path.split('\\')[-1]
# # name, ext = os.path.splitext(f_name)
    
# # img = dcm_2_jpg(sample_dcm_path)
# # new_img = win_scale(np.array(img), 1796, 3968, np.double, ([0, 255]))

# # cv2.imshow('aa', new_img)
# # cv2.waitKey()
# # cv2.destroyAllWindows()

# # # img.show()

# sample_nii_path = lung_abn_nii_path[0]

# proxy = nib.load(sample_nii_path)

# arr = np.array(proxy.get_fdata()).T[0,:,:]

# width, height = arr.shape

# out = np.empty((width, height, 3), dtype=np.uint8)
# out[:,:,0] = arr
# out[:,:,1] = arr
# out[:,:,2] = arr

# print(out.shape)

# cv2.imwrite(os.path.join('..', 'JPG', 'sample_0t.jpg'), out)

In [ ]:
# sample_dcm_path = lung_abn_dcm_path[-1]

# arr = dcm_2_jpg(sample_dcm_path)

# # print(ds.PhotometricInterpretation)

# cv2.imshow('arr', arr)
# cv2.waitKey()
# cv2.destroyAllWindows()




